# 4. Reflection

## Overview

The Reflection pattern introduces self-evaluation capabilities to agent systems. This architecture enables agents to:

- **Analyze their own outputs** for quality and completeness
- **Make iterative improvements** by regenerating responses
- **Ensure requirements are met** before finalizing responses
- **Implement quality control** through automated review cycles

![Reflection Architecture](../docs/reflection.png)

### Key Components

1. **Generator Agent**: Creates initial responses to user queries
2. **Reflection Agent**: Evaluates the quality and completeness of responses
3. **Feedback Loop**: Allows for regeneration based on reflection results
4. **Quality Criteria**: Structured evaluation framework

This pattern is particularly useful for:
- **Complex reasoning tasks** requiring multiple iterations
- **Content creation** with quality standards
- **Customer service** where accuracy is critical
- **Educational applications** with learning feedback

## Implementation Steps

### Step 1: Import Required Dependencies

We start by importing the necessary libraries for implementing the reflection pattern:
- **LangChain OpenAI**: For LLM integration
- **Message types**: For structured conversation handling
- **LangGraph**: For building the reflection workflow with conditional routing

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from typing import Literal
from langgraph.types import Command
from langgraph.graph import END

### Step 2: Initialize the Language Model

We configure the Azure OpenAI model that will serve both as the **generator** and **reflection agent**.

In [ ]:
llm = AzureChatOpenAI(model="gpt-4.1-mini")

### Step 3: Create the Generator Agent

The **travel_advice** function acts as our content generator. It:
- Receives the conversation state
- Generates travel advice based on user input
- Returns the response to be evaluated by the reflection agent

In [ ]:
def travel_advice(state) -> str:
    print("Node: Travel Advice")
    response = llm.invoke(input=state["messages"])

    return {
        "messages": [response]
    }

### Step 4: Define Reflection Output Structure

We use Pydantic to define the structured output format for our reflection agent. This ensures:
- **Consistent evaluation criteria**: Each topic is checked systematically
- **Boolean feedback**: Clear yes/no answers for content inclusion
- **Topic tracking**: Identifies specific areas that need improvement

In [ ]:
from pydantic import BaseModel
class ReflectionOutput(BaseModel):
    """You reflect on the conversation and checks if input contains the requested topic."""
    topic: str
    contains_info: bool

### Step 5: Implement the Reflection Agent

The **reflection** function is the heart of our quality control system. It:

1. **Analyzes the generated content** against predefined quality criteria
2. **Checks for completeness** across multiple topics (sports, cultural, historical)
3. **Makes routing decisions**:
   - **Continue to generator**: If content is missing required topics
   - **End workflow**: If all criteria are met
4. **Provides feedback**: Specific instructions for improvement

In [ ]:
def reflection(state) -> Command[Literal["travel_advice", "__end__"]]:
    print("Node: Reflection")
    last_message = state["messages"][-1]

    structured_llm = llm.with_structured_output(ReflectionOutput)

    topic_check_list = ["sports", "cultural", "historical"]
    validations = []
    for topic in topic_check_list:
        response = structured_llm.invoke(input=[
            SystemMessage(content=f"You are a content expert, and validate if a messages contains information about topic \"{topic}\"?"),
            last_message
        ])
        validations.append(response)

    missing_topics = [response for response in validations if not response.contains_info]
    print(f"Missing topics: {[response.topic for response in missing_topics]}")

    if len(missing_topics) > 0:
        return Command(
            goto="travel_advice",
            update={"messages": [AIMessage(content="Extend output with information about following topics: " 
                                           + ", ".join([response.topic for response in missing_topics]))]}
        )
    else:
        return Command(
            goto=END
        )

### Step 6: Build the Reflection Workflow

We construct a StateGraph that defines the reflection pattern workflow:
- **Nodes**: travel_advice (generator) and reflection (evaluator)
- **Edges**: Define the flow between generation and evaluation
- **Conditional routing**: Reflection agent determines next steps

In [ ]:
from langgraph.graph import StateGraph, START
from langgraph.graph import MessagesState

builder = StateGraph(MessagesState)

builder.add_node("travel_advice", travel_advice)
builder.add_node("reflection", reflection)

builder.add_edge(START, "travel_advice")
builder.add_edge("travel_advice", "reflection")

graph = builder.compile()

### Step 7: Visualize the Workflow

Generate a visual representation of our reflection pattern to understand the flow between components.

In [ ]:
from IPython.display import Image
Image(graph.get_graph().draw_mermaid_png())

### Step 8: Test the Reflection Pattern

Let's test our reflection system with a simple travel query. The system will:
1. Generate initial travel advice
2. Evaluate content completeness
3. Request improvements if needed
4. Iterate until all criteria are met

In [ ]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that gives travel advice based on the user's input."
    ),
    HumanMessage(
        content="Perth"
    )
]

### Step 9: Execute and Monitor the Reflection Loop

Run the reflection workflow and observe how the system:
- **Generates content**
- **Evaluates quality**
- **Provides feedback**
- **Iteratively improves**

Watch for the feedback loop in action as the system refines its response.

In [ ]:
for msg in messages:
    msg.pretty_print()

async for event in graph.astream(input={"messages": messages}, stream_mode="updates"):
    for node in graph.nodes.keys():
        node_output = event.get(node, {})
        if node_output is not None:
            output_msgs = node_output.get("messages", [])
            for msg in output_msgs:
                msg.pretty_print()

## Key Takeaways

### Benefits of the Reflection Pattern
- **Quality Assurance**: Automated content evaluation ensures consistent standards
- **Iterative Improvement**: Continuous refinement until criteria are met
- **Structured Feedback**: Clear, actionable improvement suggestions
- **Scalable Evaluation**: Easily extend evaluation criteria for different domains

### When to Use Reflection
- **Content Creation**: Blog posts, documentation, reports requiring completeness
- **Customer Support**: Ensuring responses address all customer concerns
- **Educational Content**: Verifying learning objectives are covered
- **Quality Control**: Any scenario where output standards matter

### Performance Considerations
- **Cost**: Multiple LLM calls increase operational costs
- **Latency**: Reflection loops add processing time
- **Stopping Criteria**: Set maximum iterations to prevent infinite loops

### Next Steps
Consider combining reflection with other patterns:
- **Routing + Reflection**: Route to specialized agents with quality control
- **Tool Calling + Reflection**: Verify tool usage and results
- **Parallelism + Reflection**: Concurrent generation with centralized evaluation